In [ ]:
import cv2
import os
import glob
import numpy as np

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            return None

        largest_face = max(faces, key=lambda f: f[2] * f[3])
        x, y, w, h = largest_face
        return img[y:y+h, x:x+w]

    folder = "dataa"
    if not os.path.exists(folder):
        os.makedirs(folder)

    existing_users = glob.glob(f"{folder}/user.*.*.jpg")
    user_ids = set()

    for f in existing_users:
        parts = os.path.basename(f).split('.')
        if len(parts) == 4 and parts[1].isdigit():
            user_ids.add(int(parts[1]))

    new_user_id = max(user_ids, default=0) + 1

    print(f"Starting with user ID: {new_user_id}")
    print("Resuming from image 1")

    # Attempt to open camera with retry logic
    for attempt in range(3):  # Retry 3 times if the camera fails
        cap = cv2.VideoCapture(0)
        if cap.isOpened():
            break
        print(f"Error: Unable to access the camera. Retrying {attempt + 1}/3...")
        cap.release()

    if not cap.isOpened():
        print("Error: Camera could not be opened. Exiting...")
        return

    print("Collecting samples... Look at the camera!")

    img_id = 0  
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
            continue  # Skip this iteration and retry

        face = face_cropped(frame)
        if face is not None:
            img_id += 1
            face = cv2.resize(face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = f"{folder}/user.{new_user_id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)

            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped face", face)

        if cv2.waitKey(1) == 13 or img_id >= 200:
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Collecting samples for user {new_user_id} completed!")

generate_dataset()


In [11]:

# ============================
# 2️⃣ TRAINING CLASSIFIER
# ============================

def train_classifier(dataa_dir):
    path = [os.path.join(dataa_dir, f) for f in os.listdir(dataa_dir)]
    faces, ids = [], []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")

# Train classifier
train_classifier("dataa")



In [14]:

# ============================
# 3️⃣ FACE RECOGNITION
# ============================
import cv2
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        
        id, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))
        
        print(f"Detected ID: {id}, Confidence: {confidence}")

        if confidence > 60:
            if id == 1:
                name = "Lokendra"
            elif id == 2:
                name = "khusi"
            elif id == 3:
                name = "Mr yunis lone"               
            else:
                name = "Unknown"
            cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0) if confidence > 60 else (0, 0, 255), 2, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

    return img

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)
if not video_capture.isOpened():
    print("Error: Unable to access the camera.")
    exit()

print("Press 'Enter' to exit...")
while True:
    ret, img = video_capture.read()
    if not ret:
        print("Failed to capture frame.")
        break

    img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), clf)
    cv2.imshow("Face Detection", img)

    if cv2.waitKey(1) == 13:
        break

video_capture.release()
cv2.destroyAllWindows()


Press 'Enter' to exit...
Detected ID: 1, Confidence: 70
Detected ID: 1, Confidence: 70
Detected ID: 1, Confidence: 71
Detected ID: 1, Confidence: 71
Detected ID: 1, Confidence: 70
Detected ID: 1, Confidence: 71
Detected ID: 1, Confidence: 70
Detected ID: 1, Confidence: 71
Detected ID: 1, Confidence: 70
Detected ID: 1, Confidence: 72
Detected ID: 1, Confidence: 70
Detected ID: 1, Confidence: 70
Detected ID: 1, Confidence: 70
Detected ID: 1, Confidence: 71
Detected ID: 1, Confidence: 71
Detected ID: 1, Confidence: 70
Detected ID: 1, Confidence: 71
Detected ID: 1, Confidence: 71
Detected ID: 1, Confidence: 72
Detected ID: 1, Confidence: 71
Detected ID: 1, Confidence: 72
Detected ID: 1, Confidence: 72
Detected ID: 1, Confidence: 72
Detected ID: 1, Confidence: 73
Detected ID: 1, Confidence: 74
Detected ID: 1, Confidence: 75
Detected ID: 1, Confidence: 74
Detected ID: 1, Confidence: 75
Detected ID: 1, Confidence: 75
Detected ID: 1, Confidence: 75
Detected ID: 1, Confidence: 75
Detected ID: 1

In [8]:
#location pre gui not gui
import cv2
import numpy as np
import requests

# Load Haar Cascade classifier and trained recognizer
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Function to get location (IP-based) with latitude and longitude
def get_location():
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        loc = data.get("loc", "Unknown")
        latitude, longitude = loc.split(",") if loc != "Unknown" else ("Unknown", "Unknown")
        return latitude, longitude, data.get("city", "Unknown City"), data.get("region", "Unknown Region"), data.get("country", "Unknown Country")
    except Exception as e:
        print("Error getting location:", e)
        return "Unknown", "Unknown", "Unknown", "Unknown", "Unknown"

# Function to send data to an API (can be a webhook or your own server)
def send_location_data(person_name, confidence, location):
    payload = {
        "person": person_name,
        "confidence": confidence,
        "location": f"Lat: {location[0]}, Lon: {location[1]}, {location[2]}, {location[3]}, {location[4]}"
    }
    try:
        response = requests.post("https://your-api-endpoint.com/send_location", json=payload)
        if response.status_code == 200:
            print("Location sent successfully!")
        else:
            print("Failed to send location, status code:", response.status_code)
    except Exception as e:
        print("Error sending location:", e)

# Function to recognize faces and send location only once on high confidence
sent_location = False

def recognize_and_send(img, classifier, scaleFactor, minNeighbors, color, clf):
    global sent_location
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        
        id, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))

        if confidence > 60:
            if id == 1:
                name = "Lokendra"
            elif id == 2:
                name = "lokendra"
            elif id == 3:
                name = "archie"
            elif id == 4:
                name = " Mr. yunis lone"
            elif id == 5:
                name = " sanu"
            elif id == 6:
                name = "santoshi"
            elif id == 7:
                name = "kk saraswat"
            elif id == 8:
                name = "chikara"
            elif id == 9:
                name = "abhisekh"  
                
            elif id == 10:
                name = "krishna" 
            else:
                name = "Unknown"
            color = (0, 255, 0) if confidence > 80 else (0, 165, 255)
            cv2.putText(img, f"{name} ({confidence}%)", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)

            # If confidence is over 80% and location hasn't been sent yet
            if confidence > 80 and name != "Unknown" and not sent_location:
                location = get_location()
                print(f"{name} recognized with confidence {confidence}%, Location: {location}")
                send_location_data(name, confidence, location)
                sent_location = True
        else:
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

    return img

# Open webcam
video_capture = cv2.VideoCapture(0)
if not video_capture.isOpened():
    print("Error: Unable to access the camera.")
    exit()

print("Press 'Enter' to exit...")
while True:
    ret, img = video_capture.read()
    if not ret:
        print("Failed to capture frame.")
        break

    img = recognize_and_send(img, faceCascade, 1.3, 6, (255, 255, 255), clf)
    cv2.imshow("Face Recognition with Location", img)

    if cv2.waitKey(1) == 13:
        break

video_capture.release()
cv2.destroyAllWindows()


Press 'Enter' to exit...
kk saraswat recognized with confidence 81%, Location: ('27.5035', '77.6722', 'Mathura', 'Uttar Pradesh', 'IN')
Error sending location: HTTPSConnectionPool(host='your-api-endpoint.com', port=443): Max retries exceeded with url: /send_location (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B856D010D0>, 'Connection to your-api-endpoint.com timed out. (connect timeout=None)'))


Lokendra recognized with confidence 85%, Location: ('27.7945', '77.4368', 'Kosi', 'Uttar Pradesh', 'IN')
